In [ ]:
from ultralytics import YOLO

from typing import Dict, Union, Tuple, List, Optional
from collections import Counter
import os

import matplotlib.pyplot as plt
import pandas as pd

# Гиперпараметры

In [ ]:
# Данные для тестирования
PATH_TO_TEST_DATASET = '../demo_dataset/images/'

# Параметры для тестирования
COMPARE_BY_COUNT = True  # Сравнивать по количеству, иначе по conf(достоверности предсказания)
TEST_PARAMS = {
    'verbose': False,
    'conf': 0.25,  # порог достоверности обнаружения объекта, default=0.25
    'iou': 0.3,  # пересечение порога объединения (IOU), default=0.7
}

Проверка GPU

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
import torch

cuda_flag = torch.cuda.is_available()
device = 'cuda' if cuda_flag else 'cpu'
device

Необходимые функции

In [ ]:
def show_pred_img(preds: list, index_show: int) -> None:
    """Результат работы модели на одной картинке"""
    img = preds[index_show].plot()
    img = Image.fromarray(img[..., ::-1])
    # display(img)
    plt.figure(figsize=(5,5))
    plt.axis('off')
    plt.imshow(img)

# Predict

In [ ]:
model = YOLO('../weights/best.pt')
preds = model.predict(PATH_TO_TEST_DATASET, device=device, **TEST_PARAMS)

In [ ]:
for i in range(min(len(preds), 5)):
    show_pred_img(preds, i)

# Ensemble

In [ ]:
from ensemble_boxes import *
import cv2
from random import randint


def draw_boxes_from_list(path_to_image, boxes, labels, colors, names):
    image = cv2.imread(path_to_image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) 
    height, width, _ = image.shape

    for object_, label_ in zip(boxes, labels):
        color = colors[int(label_ + 1)]
        x_min, y_min, x_max, y_max = object_
        x_min = int(x_min * width)
        y_min = int(y_min * height)
        x_max = int(x_max * width)
        y_max = int(y_max * height)

        image = cv2.putText(
            image, names[int(label_)], (int(x_min), int(y_min - 10)),
            fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1,
            color=color, thickness=6,
        )
        image = cv2.rectangle(image, (x_min, y_min), (x_max, y_max), color=color, thickness=3)
    return image


def restructure_preds(yolo_pred):
    """
    Формирует предскзаания моделей в необходимом формате:
    return:
        (координаты bbox, уверенность в предсказаниях, предсказанные лейблы)
    """
    boxes_list, scores_list, labels_list = [], [], []
    for object_ in yolo_pred[0].boxes:
        boxes_list.extend(object_.xyxyn.tolist())
        scores_list.extend(object_.conf.tolist())
        labels_list.extend(object_.cls.tolist())
    return boxes_list, scores_list, labels_list


def ensemble_boxes(
        models: List[YOLO],
        path_to_image: str,
        weights: Optional[List[float]] = None,
        run_type: str = 'wbf',

        iou_thr: float = 0.3,
        skip_box_thr: float = 0.25,
):
    """
    Данная функция усредняет предсказания модели по боксам, исходя из ряда параметров

    models: массив моделей, которые будут делать предсказание
    path_to_image: путь до изображения для предсказания
    weights: значимость каждой модели в ансамбле
    run_type: тип усреднения
    iou_thr: значение iou в совпадении полей
    skip_box_thr: минимальная уверенность модели в предсказании
    """
    if weights is None:
        weights = [1 for _ in range(len(models))]

    boxes_, scores_, labels_ = [], [], []
    for i, model in enumerate(models):
        yolo_model_predict = model.predict(path_to_image, device=device, verbose=False, conf=skip_box_thr, iou=iou_thr)
        boxes_list, scores_list, labels_list = restructure_preds(yolo_model_predict)

        boxes_.append(boxes_list)
        scores_.append(scores_list)
        labels_.append(labels_list)
    
    if run_type == 'wbf':
        boxes, scores, labels = weighted_boxes_fusion(
            boxes_,
            scores_,
            labels_,
            weights=weights,
            iou_thr=iou_thr,
            skip_box_thr=skip_box_thr
        )

    elif run_type == 'non_maximum_weighted':
        boxes, scores, labels = non_maximum_weighted(
            boxes_,
            scores_,
            labels_,
            weights=weights,
            iou_thr=iou_thr,
            skip_box_thr=skip_box_thr
        )

    else:
        raise NotImplementedError(f"{run_type} type method for ensembling boxes is not implemented. Available "
                                  f"methods: ['non_maximum_weighted', 'wbf']")
    return boxes, scores, labels

In [ ]:
model1 = YOLO('../weights/best.pt')
model2 = YOLO('')

models = [model1, ]
weights = [1, ]

COLORS = {name: [randint(0, 255),randint(0, 255),randint(0, 255)] for name in model1.names.keys()}
NAMES = list(model1.names.values())

for i, filename in enumerate(os.listdir(PATH_TO_TEST_DATASET)):
    path = os.path.join(PATH_TO_TEST_DATASET, filename)
    
    boxes, scores, labels = ensemble_boxes(
        models=models,
        path_to_image=path,
        weights=weights,
        run_type='wbf',  # ['non_maximum_weighted', 'wbf']
        iou_thr=TEST_PARAMS['iou'],
        skip_box_thr=TEST_PARAMS['conf'],
    )

    result_image = draw_boxes_from_list(
        path_to_image=path,
        boxes=boxes,
        labels=labels,
        colors=COLORS,
        names=NAMES,
    )

    if i < 6:
        plt.figure(figsize=(5, 5))
        plt.axis('off')
        plt.imshow(result_image)